# Hannah Stepanek - Thinking like a Panda: Everything you need to know to use pandas the right way.

YouTube link: https://www.youtube.com/watch?v=ObUcgEO4N8w

This is a collection of notes and examples taken from Hannah Stepanek's talk at PyCon 2019.

In [1]:
import numpy as np
import pandas as pd

## MultiIndex DataFrame
-------------

This representation in memory allows for more efficient compression; The same values don't have to be repeated over and over in memory.

In [2]:
# This is a CSV of restaurant ratings with anonymized users and businesses
df = pd.read_csv('~/data/rating_final.csv', index_col=[0,1])
df.head()

rating  food_rating  service_rating
userID placeID                                     
U1077  135085        2            2               2
       135038        2            2               1
       132825        2            2               2
       135060        1            2               2
U1068  135104        1            1               2

Check the size of the MultiIndex df in memory:

In [3]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1161 entries, (U1077, 135085) to (U1068, 132660)
Data columns (total 3 columns):
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(3)
memory usage: 41.2 KB


Let's compare that to reading the CSV file normally:

In [4]:
big_df = pd.read_csv('~/data/rating_final.csv')
big_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1161 entries, 0 to 1160
Data columns (total 5 columns):
userID            1161 non-null object
placeID           1161 non-null int64
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(4), object(1)
memory usage: 106.7 KB


In [5]:
big_df.head()

,userID,placeID,rating,food_rating,service_rating
0,U1077,135085,2,2,2
1,U1077,135038,2,2,1
2,U1077,132825,2,2,2
3,U1077,135060,1,2,2
4,U1068,135104,1,1,2


`pd.DataFrame.set_index()` can also set a MultiIndex, and `inplace=True` is to prevent copying the data in memory if possible, again, for efficiency.

This method can be useful if you don't know the shape of the data beforehand, or need to perform another operation before setting the index.

In [6]:
big_df.set_index(keys=['userID', 'placeID'], inplace=True)
big_df.head()

rating  food_rating  service_rating
userID placeID                                     
U1077  135085        2            2               2
       135038        2            2               1
       132825        2            2               2
       135060        1            2               2
U1068  135104        1            1               2

What's the size of the `big_df` now?

In [7]:
big_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1161 entries, (U1077, 135085) to (U1068, 132660)
Data columns (total 3 columns):
rating            1161 non-null int64
food_rating       1161 non-null int64
service_rating    1161 non-null int64
dtypes: int64(3)
memory usage: 41.2 KB


Originally, `big_df` was **106.7 KB** and by utilizing the MultiIndex its now **41.3 KB**.

## Loading and Normalizing the Data
----------------

In [8]:
# A different CSV of NYC open data on restaurant inspections
cols = ['DBA', 'INSPECTION DATE', 'SCORE']
data = pd.read_csv('~/data/New_York_City_Restaurant_Inspection_Results.csv', usecols=cols)
data.head()

,DBA,INSPECTION DATE,SCORE
0,MORRIS PARK BAKE SHOP,05/11/2018,5.0
1,MORRIS PARK BAKE SHOP,05/11/2018,5.0
2,MORRIS PARK BAKE SHOP,05/18/2017,7.0
3,MORRIS PARK BAKE SHOP,05/18/2017,7.0
4,MORRIS PARK BAKE SHOP,02/18/2016,10.0


In [9]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383791 entries, 0 to 383790
Data columns (total 3 columns):
DBA                383236 non-null object
INSPECTION DATE    383791 non-null object
SCORE              365713 non-null float64
dtypes: float64(1), object(2)
memory usage: 54.4 MB


**How to go about normalizing this data?**

In [10]:
data.isnull().sum()

DBA                  555
INSPECTION DATE        0
SCORE              18078
dtype: int64

1. Wrangle the NaNs when the data is loaded into memory
2. Pass in the applicable dtypes to columns

In [11]:
# This code was in the slide, but the type casting isn't playing nicely for some reason... 
# data = pd.read_csv('~/data/New_York_City_Restaurant_Inspection_Results.csv', usecols=cols, 
#                    na_values=['Not Applicable', ''], keep_default_na=False,
#                    dtype={'DBA': str, 'INSPECTION DATE': datetime, 'SCORE': np.int8})

data = pd.read_csv('~/data/New_York_City_Restaurant_Inspection_Results.csv', usecols=cols, 
                   na_values=['Not Applicable', '', np.NaN, ' ', ], keep_default_na=False,)

data['DBA'] = data['DBA'].astype(str, copy=False)
data['INSPECTION DATE'] = pd.to_datetime(data['INSPECTION DATE'], infer_datetime_format=True, format="%y/%m/%d", cache=True)

data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 383791 entries, 0 to 383790
Data columns (total 3 columns):
DBA                383791 non-null object
INSPECTION DATE    383791 non-null datetime64[ns]
SCORE              365713 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 32.8 MB


In [12]:
data.set_index(keys=['DBA', 'INSPECTION DATE'], inplace=True)

In [13]:
data.head(10)

SCORE
DBA                   INSPECTION DATE       
MORRIS PARK BAKE SHOP 2018-05-11         5.0
                      2018-05-11         5.0
                      2017-05-18         7.0
                      2017-05-18         7.0
                      2016-02-18        10.0
                      2016-02-18        10.0
WENDY'S               2019-03-04        13.0
                      2019-03-04        13.0
                      2019-03-04        13.0
                      2018-03-13        12.0

In [14]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 383791 entries, (MORRIS PARK BAKE SHOP, 2018-05-11 00:00:00) to (nan, 1900-01-01 00:00:00)
Data columns (total 1 columns):
SCORE    365713 non-null float64
dtypes: float64(1)
memory usage: 5.9 MB


Our `data` DataFrame shrank **from 54.4 MB to 5.9 MB!**

## Analyzing and Manipulating Data
--------------------------

In [15]:
# Indexing with boolean expressions
# https://stackoverflow.com/questions/21415661/logical-operators-for-boolean-indexing-in-pandas
data[ (data['SCORE'] >= 90) ].sort_values(by='INSPECTION DATE', ascending=False).head() 

SCORE
DBA          INSPECTION DATE       
BICKLES 2 GO 2019-05-08       164.0
GARDEN CAFE  2019-05-08       102.0
             2019-05-08       102.0
             2019-05-08       102.0
             2019-05-08       102.0

In [16]:
# iloc syntax
data.iloc[0]

SCORE    5.0
Name: (MORRIS PARK BAKE SHOP, 2018-05-11 00:00:00), dtype: float64

In [18]:
# loc syntax
data.loc[ (data['SCORE'] >= 98) ].head()

SCORE
DBA                               INSPECTION DATE       
ESPERANTO RESTAURANT BAR AND JUGO 2017-10-24        99.0
                                  2017-10-24        99.0
                                  2017-10-24        99.0
                                  2017-10-24        99.0
                                  2017-10-24        99.0

NOTE: `DataFrame.merge()` does **not** work with MultiIndex DataFrames. `DataFrame.join()` does though!

In [19]:
# Apply() Anti-Pattern
# df['mean'] = df.apply(np.mean, axis=1)
# The above method is 2x as slow as the code below

data['mean'] = data.mean(axis=1)
data.head()

SCORE  mean
DBA                   INSPECTION DATE             
MORRIS PARK BAKE SHOP 2018-05-11         5.0   5.0
                      2018-05-11         5.0   5.0
                      2017-05-18         7.0   7.0
                      2017-05-18         7.0   7.0
                      2016-02-18        10.0  10.0

**MultiIndex Groupby:**

In [20]:
total = data['SCORE'].groupby(by=['DBA']).count()
total.rename('total', inplace=True).head()

DBA
#1 Chinese Restaurant           22
#1 GARDEN CHINESE RESTAURANT    14
#1 Natural Juice Bar             8
#1 SABOR LATINO RESTAURANT      32
$ 1 SLICE PIZZA                 21
Name: total, dtype: int64

## Summary

-------------

* Avoid looping
* Normalize data
* Use MultiIndexed DataFrames
* Use inplace operations
* Take advantage of Cython as needed (custom apply functions)